In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


###### ArCo ARMADO DE LA BASE#######
import os
from typing import List
from datetime import datetime

# Establecer el directorio de trabajo
os.chdir("C:/Users/supegui/Documents/Metodología ArCo/Python/Excel")
# Saca la lista de archivos del directorio de trabajo
#os.listdir()
# Obtener la fecha actual
fecha_actual = datetime.now().strftime("%Y %m %d")
#print(fecha_actual)

# Construir el nombre del archivo basado en la fecha actual
#nombre_archivo = f"{fecha_actual} Informe ArCo.xlsx"

# Construir el nombre del archivo cuando la fecha no es la actual. Aquí solo se modifica la fecha. 
nombre_archivo = "2024 11 01 - Base Original.xlsx"

data_arco = pd.read_excel(nombre_archivo)

In [17]:
# 1. Primero, creamos una matriz de características binarias para cada instrumento
def crear_matriz_caracteristicas(data_arco):
    # Seleccionamos las columnas relevantes
    cols_usuarios = ['Emprendedores', 'Mipymes', 'Grandes empresas', 'Gobierno ', 'Academia', 'Entidades de soporte', 'Personas naturales']
    
    # Creamos una matriz donde cada fila es un instrumento y cada columna es un tipo de usuario
    matriz_caracteristicas = data_arco[cols_usuarios].copy()
    
    # Convertimos 'Si'/'No' a 1/0
    matriz_caracteristicas = (matriz_caracteristicas == 'Si').astype(int)
    
    return matriz_caracteristicas

In [18]:
matriz_cara = crear_matriz_caracteristicas(data_arco)

print(matriz_cara)

     Emprendedores  Mipymes  Grandes empresas  Gobierno   Academia  \
0                0        0                 0          0         0   
1                0        0                 1          0         0   
2                0        1                 1          1         1   
3                0        0                 0          1         0   
4                0        0                 0          1         1   
..             ...      ...               ...        ...       ...   
414              0        0                 0          0         0   
415              0        1                 0          1         1   
416              0        1                 0          1         1   
417              0        0                 0          0         0   
418              1        1                 1          1         0   

     Entidades de soporte  Personas naturales  
0                       1                   0  
1                       1                   0  
2              

In [19]:

"""
# 2. Calculamos la similitud entre instrumentos - Jaccard
def calcular_similitud(data_arco):
    # Obtenemos la matriz de características
    matriz = crear_matriz_caracteristicas(data_arco)
    
    # Inicializamos la matriz de similitud
    n_instrumentos = len(data_arco)
    matriz_similitud = np.zeros((n_instrumentos, n_instrumentos))
    
    # Calculamos la similitud para cada par de instrumentos
    for i in range(n_instrumentos):
        for j in range(n_instrumentos):
            # Extraemos los vectores de características para los instrumentos i y j
            instrumento_i = matriz.iloc[i].values
            instrumento_j = matriz.iloc[j].values
            
            # Calculamos similitud usando coincidencias
            coincidencias = sum(instrumento_i & instrumento_j)  # AND lógico
            total_usuarios = sum(instrumento_i | instrumento_j)  # OR lógico
            
            # Calculamos el coeficiente de Jaccard
            if total_usuarios > 0:
                similitud = coincidencias / total_usuarios
            else:
                similitud = 0
                
            matriz_similitud[i][j] = similitud
    
    return matriz_similitud
""""2024 10 25 - Base Original.xlsx"
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calcular_similitud(data_arco):
    # Obtenemos la matriz de características
    matriz = crear_matriz_caracteristicas(data_arco)
    
    # Calculamos la matriz de similitud de coseno
    matriz_similitud = cosine_similarity(matriz)
    
    return matriz_similitud


In [21]:
# 3. Convertimos la similitud a ratings
def calcular_ratings(data_arco):
    # Calculamos la matriz de similitud
    matriz_similitud = calcular_similitud(data_arco)
    
    # Creamos un DataFrame con los ratings
    ratings = []
    
    for i in range(len(matriz_similitud)):
        for j in range(len(matriz_similitud)):
            if i != j:  # No incluimos la similitud de un instrumento consigo mismo
                ratings.append({
                    'Instrumento_A': data_arco.iloc[i]['Código'],
                    'Nombre_A': data_arco.iloc[i]['Nombre del Instrumento'],
                    'Instrumento_B': data_arco.iloc[j]['Código'],
                    'Nombre_B': data_arco.iloc[j]['Nombre del Instrumento'],
                    'Rating': round(matriz_similitud[i][j] * 5, 2)  # Convertimos a escala 0-5
                })
    
    df_ratings = pd.DataFrame(ratings)
    
    # Ordenamos por rating de mayor a menor
    df_ratings = df_ratings.sort_values('Rating', ascending=False)
    
    return df_ratings

In [22]:
# 4. Función principal que integra todo el proceso
def generar_ratings_colaborativos(data_arco):
    # Calculamos los ratings
    df_ratings = calcular_ratings(data_arco)
    
    # Filtramos solo los pares con rating significativo (por ejemplo, mayor a 2)
    df_ratings_significativos = df_ratings[df_ratings['Rating'] > 2]
    
    return df_ratings_significativos

# Ejemplo de uso:
ratings_finales = generar_ratings_colaborativos(data_arco)
print(ratings_finales.head(10))

        Instrumento_A                                           Nombre_A  \
132990           2853                      Programa Incentivo a medallas   
129027           2832  Beneficio de Estímulo Económico de Sometimient...   
128984           2832  Beneficio de Estímulo Económico de Sometimient...   
128986           2832  Beneficio de Estímulo Económico de Sometimient...   
128991           2832  Beneficio de Estímulo Económico de Sometimient...   
128992           2832  Beneficio de Estímulo Económico de Sometimient...   
128994           2832  Beneficio de Estímulo Económico de Sometimient...   
128997           2832  Beneficio de Estímulo Económico de Sometimient...   
162672           3631  Acompañamiento para la territorialización de l...   
129007           2832  Beneficio de Estímulo Económico de Sometimient...   

        Instrumento_B                                           Nombre_B  \
132990            552  Servicio de adjudicación de bienes fiscales pa...   
129027     

## Sistema de recomendación


In [23]:
from surprise import Dataset, Reader, SVD, KNNBasic, NMF, accuracy, similarities
from surprise.model_selection import train_test_split, cross_validate
from collections import defaultdict

class SistemaRecomendacion:
    def __init__(self, df_ratings):
        """
        Inicializa el sistema de recomendación con el DataFrame de ratings
        
        Parameters:
        -----------
        df_ratings : pandas.DataFrame
            DataFrame que debe contener las columnas ['Instrumento_A', 'Instrumento_B', 'Rating']
        """
        # Validar que el DataFrame tiene las columnas necesarias
        required_columns = ['Instrumento_A', 'Instrumento_B', 'Rating']
        if not all(col in ratings_finales.columns for col in required_columns):
            raise ValueError(f"El DataFrame debe contener las columnas: {required_columns}")
            
        self.ratings_finales = ratings_finales.copy()
        
        # Verificar que no hay valores nulos
        if self.ratings_finales[required_columns].isna().any().any():
            raise ValueError("El DataFrame contiene valores nulos")
        
        # Aseguramos que los ratings están en el rango correcto
        self.ratings_finales['Rating'] = self.ratings_finales['Rating'].clip(0, 5)
        
        # Configuramos el reader con un rango válido
        self.reader = Reader(rating_scale=(0, 5))
        
        try:
            # Convertimos a formato Surprise
            self.data = Dataset.load_from_df(
                self.ratings_finales[required_columns], 
                self.reader
            )
        except Exception as e:
            raise ValueError(f"Error al convertir datos al formato Surprise: {str(e)}")
        
        # Definimos los modelos base
        self.modelos = {
            'SVD': SVD(
                n_factors=50,
                n_epochs=20,
                lr_all=0.005,
                reg_all=0.02,
                random_state=42
            ),
            'NMF': NMF(
                n_factors=15,
                n_epochs=50,
                random_state=42
            )
        }
        
        # Calcular el número de ratings por instrumento de manera correcta
        ratings_a = self.ratings_finales['Instrumento_A'].value_counts()
        ratings_b = self.ratings_finales['Instrumento_B'].value_counts()
        
        # Combinar los conteos
        todos_instrumentos = set(ratings_a.index) | set(ratings_b.index)
        ratings_totales = pd.Series(0, index=todos_instrumentos)
        
        for instrumento in todos_instrumentos:
            total = (ratings_a.get(instrumento, 0) + 
                    ratings_b.get(instrumento, 0))
            ratings_totales[instrumento] = total
        
        min_ratings = ratings_totales.min()
        print(f"Número mínimo de ratings por instrumento: {min_ratings}")
        print(f"Número máximo de ratings por instrumento: {ratings_totales.max()}")
        print(f"Promedio de ratings por instrumento: {ratings_totales.mean():.2f}")
        
        # Agregamos KNN solo si tenemos suficientes datos y ratings
        if len(self.ratings_finales) > 10 and min_ratings >= 2:
            # Calculamos k de manera más conservadora
            k = min(
                20,  # máximo valor de k
                max(2, len(self.ratings_finales) // 10)  # k proporcional al tamaño del dataset
            )
            
            self.modelos['KNN'] = KNNBasic(
                k=k,
                min_k=2,
                sim_options={
                    'name': 'pearson_baseline',
                    'user_based': False,
                    'min_support': 2,
                    'shrinkage': 100
                }
            )
            print(f"KNN configurado con k={k}")
        else:
            print("No se incluirá KNN debido a insuficientes datos o ratings por instrumento")
        
        self.mejor_modelo = None
        self.metricas_modelos = {}
        
        print(f"\nTotal de instrumentos únicos: {len(todos_instrumentos)}")
        print(f"Total de ratings: {len(self.ratings_finales)}")

    def evaluar_modelos(self):
        """
        Evalúa diferentes modelos usando validación cruzada
        
        Returns:
        --------
        dict
            Diccionario con las métricas de evaluación de cada modelo
        """
        if len(self.ratings_finales) < 5:
            raise ValueError("Se necesitan al menos 5 ratings para evaluar los modelos")
            
        print("\nEvaluando modelos...")
        for nombre, modelo in self.modelos.items():
            try:
                # Realizar validación cruzada
                n_folds = min(5, len(self.ratings_finales) // 2)
                n_folds = max(2, n_folds)
                
                resultados = cross_validate(
                    modelo, 
                    self.data, 
                    measures=['RMSE', 'MAE'], 
                    cv=n_folds, 
                    verbose=False
                )
                
                self.metricas_modelos[nombre] = {
                    'RMSE_medio': np.mean(resultados['test_rmse']),
                    'MAE_medio': np.mean(resultados['test_mae']),
                    'tiempo_fit_medio': np.mean(resultados['fit_time']),
                    'tiempo_test_medio': np.mean(resultados['test_time'])
                }
                
                print(f"✓ Modelo {nombre} evaluado exitosamente")
                print(f"  RMSE: {self.metricas_modelos[nombre]['RMSE_medio']:.4f}")
                print(f"  MAE: {self.metricas_modelos[nombre]['MAE_medio']:.4f}")
                
            except Exception as e:
                print(f"✗ Error evaluando modelo {nombre}: {str(e)}")
                print("  Este modelo será excluido de las recomendaciones")
                continue
        
        if not self.metricas_modelos:
            raise ValueError("Ningún modelo pudo ser evaluado exitosamente")
            
        # Encontrar el mejor modelo basado en RMSE
        mejor_modelo_nombre = min(
            self.metricas_modelos, 
            key=lambda x: self.metricas_modelos[x]['RMSE_medio']
        )
        self.mejor_modelo = self.modelos[mejor_modelo_nombre]
        print(f"\n→ Mejor modelo: {mejor_modelo_nombre}")
        
        return self.metricas_modelos

    def evaluar_modelos(self):
        """
        Evalúa diferentes modelos usando validación cruzada
        
        Returns:
        --------
        dict
            Diccionario con las métricas de evaluación de cada modelo
        """
        if len(self.ratings_finales) < 5:
            raise ValueError("Se necesitan al menos 5 ratings para evaluar los modelos")
            
        print("\nEvaluando modelos...")
        for nombre, modelo in self.modelos.items():
            try:
                # Realizar validación cruzada
                n_folds = min(5, len(self.ratings_finales) // 2)
                n_folds = max(2, n_folds)
                
                resultados = cross_validate(
                    modelo, 
                    self.data, 
                    measures=['RMSE', 'MAE'], 
                    cv=n_folds, 
                    verbose=False
                )
                
                self.metricas_modelos[nombre] = {
                    'RMSE_medio': np.mean(resultados['test_rmse']),
                    'MAE_medio': np.mean(resultados['test_mae']),
                    'tiempo_fit_medio': np.mean(resultados['fit_time']),
                    'tiempo_test_medio': np.mean(resultados['test_time'])
                }
                
                print(f"✓ Modelo {nombre} evaluado exitosamente")
                print(f"  RMSE: {self.metricas_modelos[nombre]['RMSE_medio']:.4f}")
                print(f"  MAE: {self.metricas_modelos[nombre]['MAE_medio']:.4f}")
                
            except Exception as e:
                print(f"✗ Error evaluando modelo {nombre}: {str(e)}")
                print("  Este modelo será excluido de las recomendaciones")
                continue
        
        if not self.metricas_modelos:
            raise ValueError("Ningún modelo pudo ser evaluado exitosamente")
            
        # Encontrar el mejor modelo basado en RMSE
        mejor_modelo_nombre = min(
            self.metricas_modelos, 
            key=lambda x: self.metricas_modelos[x]['RMSE_medio']
        )
        self.mejor_modelo = self.modelos[mejor_modelo_nombre]
        print(f"\n→ Mejor modelo: {mejor_modelo_nombre}")
        
        return self.metricas_modelos

    def entrenar_mejor_modelo(self):
        """
        Entrena el mejor modelo con todos los datos
        """
        if self.mejor_modelo is None:
            raise ValueError("Debe evaluar los modelos primero")
            
        trainset = self.data.build_full_trainset()
        self.mejor_modelo.fit(trainset)

    def generar_recomendaciones(self, instrumento_id, n=5):
        """
        Genera recomendaciones para un instrumento específico con manejo de errores
        """
        if self.mejor_modelo is None:
            raise ValueError("Debe entrenar el modelo primero")
            
        # Obtener todos los instrumentos únicos
        instrumentos_unicos = set(self.ratings_finales['Instrumento_A'].unique()) | \
                            set(self.ratings_finales['Instrumento_B'].unique())
        
        if instrumento_id not in instrumentos_unicos:
            raise ValueError(f"Instrumento {instrumento_id} no encontrado en el conjunto de datos")
        
        # Generar predicciones para todos los pares posibles
        predicciones = []
        for otro_instrumento in instrumentos_unicos:
            if otro_instrumento != instrumento_id:
                try:
                    pred = self.mejor_modelo.predict(instrumento_id, otro_instrumento)
                    predicciones.append({
                        'Instrumento_origen': instrumento_id,
                        'Instrumento_recomendado': otro_instrumento,
                        'Rating_predicho': pred.est,
                        'Detalles': pred.details
                    })
                except Exception as e:
                    print(f"Error prediciendo para {otro_instrumento}: {str(e)}")
                    continue
        
        if not predicciones:
            raise ValueError("No se pudieron generar predicciones")
            
        # Convertir a DataFrame y ordenar
        df_recomendaciones = pd.DataFrame(predicciones)
        df_recomendaciones = df_recomendaciones.sort_values(
            'Rating_predicho', 
            ascending=False
        ).head(n)
        
        return df_recomendaciones

    def mostrar_metricas(self):
        """
        Muestra las métricas de evaluación de todos los modelos
        """
        if not self.metricas_modelos:
            raise ValueError("No hay métricas disponibles. Debe evaluar los modelos primero")
            
        df_metricas = pd.DataFrame(self.metricas_modelos).round(4)
        return df_metricas.transpose()

def crear_y_evaluar_sistema(ratings_finales):
    """
    Función principal para crear y evaluar el sistema de recomendación
    """
    # Verificar que tenemos suficientes datos
    if len(ratings_finales) < 5:
        raise ValueError("Se necesitan al menos 5 ratings para crear el sistema")
        
    # Verificar que tenemos variabilidad en los ratings
    if ratings_finales['Rating'].std() < 0.1:
        raise ValueError("No hay suficiente variabilidad en los ratings")
        
    # Crear sistema
    sistema = SistemaRecomendacion(ratings_finales)
    
    # Evaluar modelos
    try:
        metricas = sistema.evaluar_modelos()
        print("\nMétricas de evaluación:")
        print(sistema.mostrar_metricas())
        
        # Entrenar mejor modelo
        sistema.entrenar_mejor_modelo()
        
    except Exception as e:
        print(f"Error en la evaluación del sistema: {str(e)}")
        raise
    
    return sistema

# Ejemplo de uso:
"""
try:
    # Crear y evaluar el sistema
    sistema = crear_y_evaluar_sistema(ratings_finales)
    
    # Generar recomendaciones para un instrumento específico
    instrumento_ejemplo = ratings_finales['Instrumento_A'].iloc[126]
    recomendaciones = sistema.generar_recomendaciones(instrumento_ejemplo, n=20)
    print("\nRecomendaciones para instrumento", instrumento_ejemplo)
    print(recomendaciones)
    
except Exception as e:
    print(f"Error: {str(e)}")
    
"""

'\ntry:\n    # Crear y evaluar el sistema\n    sistema = crear_y_evaluar_sistema(ratings_finales)\n    \n    # Generar recomendaciones para un instrumento específico\n    instrumento_ejemplo = ratings_finales[\'Instrumento_A\'].iloc[126]\n    recomendaciones = sistema.generar_recomendaciones(instrumento_ejemplo, n=20)\n    print("\nRecomendaciones para instrumento", instrumento_ejemplo)\n    print(recomendaciones)\n    \nexcept Exception as e:\n    print(f"Error: {str(e)}")\n    \n'

In [ ]:
# Crear y entrenar el sistema de recomendación
sistema = SistemaRecomendacion(ratings_finales)

#sistema.evaluar_modelos()

Número mínimo de ratings por instrumento: 298
Número máximo de ratings por instrumento: 804
Promedio de ratings por instrumento: 586.13
KNN configurado con k=20

Total de instrumentos únicos: 412
Total de ratings: 120742

Evaluando modelos...
✓ Modelo SVD evaluado exitosamente
  RMSE: 0.2099
  MAE: 0.1222
✓ Modelo NMF evaluado exitosamente
  RMSE: 0.3565
  MAE: 0.2321
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
✓ Modelo KNN evaluado exitosamente
  RMSE: 0

{'SVD': {'RMSE_medio': 0.20991074633778362,
  'MAE_medio': 0.12221521253009289,
  'tiempo_fit_medio': 0.4337204933166504,
  'tiempo_test_medio': 0.07608060836791992},
 'NMF': {'RMSE_medio': 0.356488879812866,
  'MAE_medio': 0.23212276619549055,
  'tiempo_fit_medio': 0.8814755439758301,
  'tiempo_test_medio': 0.08737149238586425},
 'KNN': {'RMSE_medio': 0.2609552382475045,
  'MAE_medio': 0.14347850728380682,
  'tiempo_fit_medio': 1.0844550132751465,
  'tiempo_test_medio': 2.0849525928497314}}

In [25]:
sistema.entrenar_mejor_modelo()

## Gradio


In [11]:
!pip install gradio

   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   ------------- -------------------------- 6.3/18.1 MB 32.1 MB/s eta 0:00:01
   --------------------------------- ------ 14.9/18.1 MB 37.5 MB/s eta 0:00:01
   ---------------------------------------- 18.1/18.1 MB 36.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 21.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ------------ --------------------------- 2.9/9.4 MB 13.9 MB/s eta 0:00:01
   ------------------------ --------------- 5.8/9.4 MB 13.5 MB/s eta 0:00:01
   -------------------------------------- - 9.2/9.4 MB 14.6 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [44]:
import gradio as gr
import pandas as pd

# Asegúrate de que `SistemaRecomendacion` y `data_arco` ya estén definidos
# Supón que `sistema` ya está creado y entrenado

# Convertir el DataFrame `data_arco` en un diccionario para una consulta rápida
descripcion_instrumentos = data_arco.set_index("Nombre del Instrumento")["Descripcion"].to_dict()

# Función para obtener recomendaciones y sus descripciones
def obtener_recomendaciones(instrumento_id):
    # Generar recomendaciones con el sistema de recomendación
    recomendaciones_df = sistema.generar_recomendaciones(instrumento_id, n=5)
    recomendaciones = recomendaciones_df["Instrumento_recomendado"].tolist()
    
    # Obtener las descripciones para cada instrumento recomendado
    descripciones = {inst: descripcion_instrumentos.get(inst, "Descripción no disponible") for inst in recomendaciones}
    
    return recomendaciones

# Función para mostrar la descripción de un instrumento específico
def mostrar_descripcion(instrumento):
    if isinstance(instrumento, list):  # Verificar que es una lista y tomar el primer elemento
        instrumento = instrumento[0] if instrumento else None
    return descripcion_instrumentos.get(instrumento, "Descripción no disponible") if instrumento else "Descripción no disponible"

# Crear interfaz Gradio
with gr.Blocks() as interfaz:
    gr.Markdown("# Sistema de Recomendación de Instrumentos de Política")
    
    # Selección del instrumento principal
    instrumento_elegido = gr.Dropdown(label="Selecciona un instrumento",
                                      choices=list(sistema.ratings_finales["Instrumento_A"].unique()))
    
    # Sección de recomendaciones
    lista_recomendaciones = gr.Dropdown(label="Instrumentos recomendados")
    descripcion_recomendada = gr.Textbox(label="Descripción del instrumento seleccionado")
    
    # Función para actualizar lista de recomendaciones al seleccionar un instrumento
    def actualizar_recomendaciones(instrumento):
        recomendaciones = obtener_recomendaciones(instrumento)
        return gr.update(choices=recomendaciones)

    # Llamada para actualizar la lista de recomendaciones
    instrumento_elegido.change(fn=actualizar_recomendaciones, inputs=[instrumento_elegido], outputs=[lista_recomendaciones])

    # Actualización de la descripción
    lista_recomendaciones.change(fn=mostrar_descripcion, inputs=[lista_recomendaciones], outputs=[descripcion_recomendada])

# Inicia la interfaz
if __name__ == "__main__":
    interfaz.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
